In [42]:
import numpy as np


data = np.load('./10_classes_embeddings.npz')
data_alex = np.load('./Alex_emd.npz')

In [43]:
emb_trainX , trainy_n , emb_testX , testy_n = data['arr_0'],data['arr_1'],data['arr_2'],data['arr_3']
emb_trainX_alex , trainy_alex , emb_testX_alex , testy_alex = data_alex['arr_0'],data_alex['arr_1'],data_alex['arr_2'],data_alex['arr_3']

In [44]:
def concatenate_face(mainData,newData):
    embed_con = np.concatenate((mainData,newData),axis=0)
    return embed_con

embed_trianX = concatenate_face(emb_trainX,emb_trainX_alex)
embed_testX = concatenate_face(emb_testX,emb_testX_alex)
trainy = concatenate_face(trainy_n,trainy_alex)
testy = concatenate_face(testy_n,testy_alex)

In [45]:
from sklearn.preprocessing import Normalizer , LabelEncoder

in_coder = Normalizer()
out_coder = LabelEncoder()

embed_trainX_norm = in_coder.fit_transform(embed_trianX)
embed_testX_norm = in_coder.transform(embed_testX)
out_coder.fit(trainy)
trainy_enc = out_coder.transform(trainy)
testy_enc = out_coder.transform(testy)

In [46]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , classification_report

model = SVC(probability=True,kernel='linear')
model.fit(embed_trainX_norm,trainy_enc)

#predict
yhat_train = model.predict(embed_trainX_norm)
yhat_test = model.predict(embed_testX_norm)

print('Train Accuracy: {:.3f}'.format(accuracy_score(trainy_enc,yhat_train)))
print('Test Accuracy: {:.3f}'.format(accuracy_score(testy_enc,yhat_test)))
print(classification_report(testy_enc,yhat_test))

Train Accuracy: 1.000
Test Accuracy: 1.000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00         4
          11       1.00      1.00      1.00         5
          12       1.00      1.00      1.00         3
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         4
          15       1.00      1.00      1.00         5

    accuracy                         

In [66]:
import cv2
from PIL import Image
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet

detector = MTCNN()
facenet_model = FaceNet()


def extract_face(filename, re_size=(160, 160)):
    image_arr = np.array(filename)
    faces = detector.detect_faces(image_arr)
    
    if len(faces) == 0:
        # No faces detected in the image
        return None
    
    x1, y1, width, height = faces[0]['box']
    x2, y2 = x1 + width, y1 + height
    face = image_arr[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(re_size)
    face_arr = np.asarray(image)
    return face_arr

def get_embedding(facenet_model, face):
    if face is None:
        return None
    
    face_pixel = face.astype('float32')
    samples = np.expand_dims(face_pixel, axis=0)
    embedd = facenet_model.embeddings(samples)
    return embedd[0]



In [68]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if ret == True:
        face = extract_face(frame)
        face_embed = get_embedding(facenet_model, face)
        if face_embed is not None:
            samples = np.expand_dims(face_embed,axis=0)
            index = model.predict(samples)
            predict_names = out_coder.inverse_transform(index)
            print(predict_names)
            cv2.imshow('output', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('d'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 151ms/step
['Alex']
1/1 [==============================] - 0s 162ms/step
['Alex']
1/1 [==============================] - 0s 174ms/step
['Alex']
1/1 [==============================] - 0s 135ms/step
['Alex']


In [65]:
cap = cv2.VideoCapture(0)
while True:
    ret , frame = cap.read()
    if ret == True:
        cv2.imshow('output',frame)
        if cv2.waitKey(1) & 0xFF == ord('d'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()